In [ ]:
!pip install --upgrade pillow transformers

In [ ]:
import pandas as pd

In [ ]:
movies=pd.read_csv('datasetUpdate.csv')

In [ ]:
# movies.head(10)

In [ ]:
# movies.describe()

In [ ]:
# movies.info()

In [ ]:
# movies.isnull().sum()

Feature Selection Part

In [ ]:
movies.columns

In [ ]:
movies=movies[['id', 'title', 'overview', 'genre']]

In [ ]:
movies

In [ ]:
movies['tags'] = movies['overview']+movies['genre']

In [ ]:
movies

In [ ]:
# movies['overview'] = movies['overview'].fillna('')

In [ ]:
new_data  = movies.drop(columns=['overview', 'genre'])

In [ ]:
# new_data = movies

In [ ]:
# new_data['overview'] = new_data['overview'].fillna('')

In [ ]:
new_data

In [ ]:
from transformers import BertTokenizer, BertModel
print(transformers.__version__)
import torch
print(torch.__version__)

In [ ]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# def get_bert_embeddings(texts):
#     # Tokenize and encode texts
#     inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Return the mean of the token embeddings for each document
#     return outputs.last_hidden_state.mean(dim=1).numpy()

def get_bert_embeddings(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

In [ ]:
# Ensure 'tags' column is properly formatted
new_data['tags'] = new_data['tags'].astype(str).fillna('')

In [ ]:
# Generate embeddings for movie tags in batches
embeddings = get_bert_embeddings(new_data['tags'].astype(str).tolist(), batch_size=32)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Compute pairwise cosine similarities
similarity = cosine_similarity(embeddings)

In [ ]:
# def recommand(movie_title):
#     if movie_title not in new_data['title'].values:
#         print("Movie not found in dataset.")
#         return

#     # Get the index of the movie that matches the title
#     index = new_data[new_data['title'] == movie_title].index[0]

#     # Get the pairwise similarity scores of all movies with that movie
#     distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector: vector[1])

#     # Print the titles of the top 5 most similar movies
#     print(f"Top 5 recommendations for '{movie_title}':")
#     for i in distance[1:6]:  # Skip the first entry as it is the movie itself
#         print(new_data.iloc[i[0]].title)


In [ ]:
# Save the similarity matrix and movie titles
import pickle

with open('similarity_matrix.pkl', 'wb') as f:
    pickle.dump(similarity, f)
with open('movie_titles.pkl', 'wb') as f:
    pickle.dump(movies['title'].tolist(), f)

In [ ]:
def recommend(movie_title, top_n=5):
    if movie_title not in movies['title'].values:
        print("Movie not found in dataset.")
        return

    # Load similarity matrix and movie titles
    with open('similarity_matrix.pkl', 'rb') as f:
        similarity = pickle.load(f)
    with open('movie_titles.pkl', 'rb') as f:
        titles = pickle.load(f)

    # Get the index of the movie
    idx = movies[movies['title'] == movie_title].index[0]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(similarity[idx]))

    # Sort the movies based on similarity scores
    sim_scores = sorted(sim_scores, reverse=True, key=lambda x: x[1])

    # Get the scores of the top_n most similar movies
    sim_scores = sim_scores[1:top_n + 1]  # Skip the first entry (the movie itself)

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top_n most similar movies
    recommended_titles = [titles[i] for i in movie_indices]
    print(f"Top {top_n} recommendations for '{movie_title}':")
    for title in recommended_titles:
        print(title)

# Example usage
recommend('Iron Man')
